In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle(r"shared/Motor_Vehicle_Collisions_-_Crashes.pkl")
df

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN
2018241,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,...,NaN,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN
2018243,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN


In [3]:
# Borough has had the highest total number of crashes reported since 2012
df['CRASH DATE_CRASH TIME'] = pd.to_datetime(df['CRASH DATE_CRASH TIME'])  

start_date = pd.to_datetime('2012-01-01')
df = df[df['CRASH DATE_CRASH TIME'] >= start_date]

borough_crash_totals = df.groupby('BOROUGH')['COLLISION_ID'].sum()
highest_crash_borough = borough_crash_totals.idxmax()

print(highest_crash_borough)

BROOKLYN


In [4]:
# Borough has the most crashes for every 100,000 people since July 2012 using population estimates

# Population estimates
population_estimates = {
    'Bronx': 1446788,
    'Brooklyn': 2648452,
    'Manhattan': 1638281,
    'Queens': 2330295,
    'Staten Island': 487155
}


df['BOROUGH'] = df['BOROUGH'].str.lower()
df['CRASH DATE_CRASH TIME'] = pd.to_datetime(df['CRASH DATE_CRASH TIME'])
df = df[df['CRASH DATE_CRASH TIME'] >= '2012-07-01']

borough_mapping = {
    'bronx': 'Bronx',
    'brooklyn': 'Brooklyn',
    'manhattan': 'Manhattan',
    'queens': 'Queens',
    'staten island': 'Staten Island'
}


df['BOROUGH'] = df['BOROUGH'].map(borough_mapping)

borough_crash_counts = df['BOROUGH'].value_counts()

crash_rates = {borough: (count / population_estimates[borough]) * 100000 for borough, count in borough_crash_counts.items()}
highest_crash_rate_borough = max(crash_rates, key=crash_rates.get)

print(highest_crash_rate_borough)
print(crash_rates)

Manhattan
{'Brooklyn': 16652.2179748774, 'Queens': 15983.255338916317, 'Manhattan': 19121.628096767283, 'Bronx': 14193.164444272415, 'Staten Island': 11966.827806345003}


In [5]:
# Proportion of accidents are attributable to the leading cause in vehicle

filtered_df = df[df['CONTRIBUTING FACTOR VEHICLE 1'] != 'Unspecified']

contributing_factor_counts = filtered_df['CONTRIBUTING FACTOR VEHICLE 1'].value_counts(normalize=True) * 100

leading_cause = contributing_factor_counts.idxmax()

proportion = contributing_factor_counts.max()

print(leading_cause)
print(proportion)

Driver Inattention/Distraction
30.417970209853078


In [6]:
# Proportion of total crashes accounted for by the top 3 causes
proportion_top_3 = contributing_factor_counts.head(3).sum()

print(proportion_top_3)

47.59805664357876


In [7]:
# Total count of accidents that involved at least one fatality
df['CRASH DATE_CRASH TIME'] = pd.to_datetime(df['CRASH DATE_CRASH TIME'])
df = df[df['CRASH DATE_CRASH TIME'].dt.year >= 2012]

fatal_accidents = df[df['NUMBER OF PERSONS KILLED'] > 0]

total_fatal_accidents = len(fatal_accidents)

print(total_fatal_accidents)

2804


In [8]:
# The average number of accidents with at least one person dead per 1000 accidents

total_accidents = len(df)

average_fatalities_per_1000 = (total_fatal_accidents / total_accidents) * 1000

print(average_fatalities_per_1000)

1.3893258747079764


In [9]:
# Proportion of accidents in the data do not have a Borough code

missing_borough_accidents = df['BOROUGH'].isna().sum()

proportion_missing_borough = missing_borough_accidents / total_accidents

print(proportion_missing_borough)

0.3110890897784957


In [10]:
# Combination of vehicles have the most number of accidents
vehicle_combinations = df.groupby(['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2']).size()

most_common_combination = vehicle_combinations.idxmax()

print(most_common_combination)

('Sedan', 'Sedan')
